In [1]:
# Process Vo data for a final size analysis by age
# Edit 17 Aug: just look at adults and children to
# Edit 25 Aug: add Cauchemez model and other updates for ONS
# Edit 7 Oct: Try to debug the sub-epidemics problem Lorenzo noticed
# Remember to atleast_2d the design matrices!

In [2]:
%matplotlib inline
import numpy as np
import scipy.stats as st
import scipy.optimize as op
import pandas as pd
from numpy import linalg as LA
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [3]:
df = pd.read_csv('./vo_data.csv')

In [4]:
# Indices of ever positive cases
posi = ((df['first_sampling'].values == 'Positive') | (df['second_sampling'].values == 'Positive'))

In [5]:
hcol = df.household_id.values
hhids = pd.unique(df.household_id)
len(hhids)

1299

In [6]:
hh_tests = []
ages = []
for hid in hhids:
    dfh = df[df.household_id == hid]
    tests = (dfh['first_sampling'].values == 'Positive') | (dfh['second_sampling'].values == 'Positive')
    aa = dfh.iloc[:,2].values
    hh_tests.append(tests)
    ages.append(aa)

In [7]:
age_gs = pd.unique(df.age_group)
age_gs.sort()
age_gs

array(['00-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70',
       '71-80', '81-90', '91+'], dtype=object)

In [8]:
nsamp = np.zeros(len(age_gs))
npos = np.zeros(len(age_gs))

In [9]:
for i, ag in enumerate(age_gs):
    dfa = df[df.age_group == ag]
    nsamp[i] = len(dfa)
    dfp = df[posi]
    dfa = dfp[dfp.age_group == ag]
    npos[i] = len(dfa)

In [10]:
# Dictionary that puts ages in categories
# 0 is reference class
as2rg = {
    '00-10' : 1,
    '11-20' : 1,
    '21-30' : 0, 
    '31-40' : 0,
    '41-50' : 0,
    '51-60' : 0,
    '61-70' : 0,
    '71-80' : 0,
    '81-90' : 0,
    '91+'   : 0,
}

In [11]:
na = max(as2rg.values())

In [12]:
Y = [] # To store outcomes
XX = [] # To store design matrices
for i in range(0,len(hhids)):
    mya = [as2rg[a] for a in ages[i]]
    m = len(mya)
    myx = np.zeros((m,na))
    myy = np.zeros(m)
    for j, a in enumerate(mya):
        if (a>0):
            myx[j,a-1] = 1
        if (hh_tests[i][j]):
            myy[j] = 1
    Y.append(myy)
    XX.append(np.atleast_2d(myx))

In [13]:
# The above processes the data - now add final size analysis; first do a run through

In [14]:
def phi(s, logtheta=0.0):
    theta = np.exp(logtheta)
    return ((1.0 + theta*s)**(-1.0/theta))

In [15]:
x = np.array([
    -3.0,
    -2.0,
    0.1,
    0.2,
    0.3,
    0.4,
    0.5,
])

In [16]:
llaL = x[0]
llaG = x[1]
logtheta = x[2]
eta = (4./np.pi)*np.arctan(x[3])
alpha = x[4:(4+na)]
beta = x[(4+na):(4+2*na)]
gamma = x[(4+2*na):]

nlv = np.zeros(len(hhids)) # Vector of negative log likelihoods
for i in range(0,len(hhids)):
    y = Y[i]
    X = XX[i]
    if np.all(y==0.0):
        nlv[i] = np.exp(llaG)*np.sum(np.exp(alpha@(X.T)))
    else:
        # Sort to go zeros then ones WLOG (could do in pre-processing)
        ii = np.argsort(y)
        y = y[ii]
        X = X[ii,:]
        q = sum(y>0)
        r = 2**q
        m = len(y)

        # Quantities that don't vary through the sum
        Bk = np.exp(-np.exp(llaG)*np.exp(alpha@(X.T)))
        laM = np.exp(llaL)*np.outer(np.exp(beta@(X.T)),np.exp(gamma@(X.T)))
        laM *= m**eta

        BB = np.zeros((r,r)) # To be the Ball matrix
        for jd in range(0,r):
            for omd in range(0,jd+1):
                jstr = format(jd,'0' + str(m) + 'b')
                omstr = format(omd,'0' + str(m) + 'b')
                j = np.array([int(jstr[x]) for x in range(0,len(jstr))])
                om = np.array([int(omstr[x]) for x in range(0,len(omstr))])
                BB[jd,omd] = 1.0/np.prod((phi((1-j)@laM,logtheta)**om)*(Bk**(1-j)))
        nlv[i] = -np.log(LA.solve(BB,np.ones(r))[-1])
        if q>2:
            break
nll = np.sum(nlv)

In [17]:
for jd in range(0,r):
    jstr = format(jd,'0' + str(m) + 'b')
    j = np.array([int(jstr[x]) for x in range(0,len(jstr))])
    for omd in range(0,jd+1):
        omstr = format(omd,'0' + str(m) + 'b')
        om = np.array([int(omstr[x]) for x in range(0,len(omstr))])
        if np.all(om<=j):
            print('({:d},{:d}) j: {}; omega: {}.'.format(jd,omd,jstr,omstr))

(0,0) j: 0000; omega: 0000.
(1,0) j: 0001; omega: 0000.
(1,1) j: 0001; omega: 0001.
(2,0) j: 0010; omega: 0000.
(2,2) j: 0010; omega: 0010.
(3,0) j: 0011; omega: 0000.
(3,1) j: 0011; omega: 0001.
(3,2) j: 0011; omega: 0010.
(3,3) j: 0011; omega: 0011.
(4,0) j: 0100; omega: 0000.
(4,4) j: 0100; omega: 0100.
(5,0) j: 0101; omega: 0000.
(5,1) j: 0101; omega: 0001.
(5,4) j: 0101; omega: 0100.
(5,5) j: 0101; omega: 0101.
(6,0) j: 0110; omega: 0000.
(6,2) j: 0110; omega: 0010.
(6,4) j: 0110; omega: 0100.
(6,6) j: 0110; omega: 0110.
(7,0) j: 0111; omega: 0000.
(7,1) j: 0111; omega: 0001.
(7,2) j: 0111; omega: 0010.
(7,3) j: 0111; omega: 0011.
(7,4) j: 0111; omega: 0100.
(7,5) j: 0111; omega: 0101.
(7,6) j: 0111; omega: 0110.
(7,7) j: 0111; omega: 0111.


In [18]:
om >= j

array([ True,  True,  True,  True])

In [19]:
def mynll(x):
    
    try: # Ideally catch the linear algebra fail directly
        llaL = x[0]
        llaG = x[1]
        logtheta = x[2]
        eta = (4./np.pi)*np.arctan(x[3])
        alpha = x[4:(4+na)]
        beta = x[(4+na):(4+2*na)]
        gamma = x[(4+2*na):]

        nlv = np.zeros(len(hhids)) # Vector of negative log likelihoods
        for i in range(0,len(hhids)):
            y = Y[i]
            X = XX[i]
            if np.all(y==0.0):
                nlv[i] = np.exp(llaG)*np.sum(np.exp(alpha@(X.T)))
            else:
                # Sort to go zeros then ones WLOG (could do in pre-processing)
                ii = np.argsort(y)
                y = y[ii]
                X = X[ii,:]
                q = sum(y>0)
                r = 2**q
                m = len(y)

                # Quantities that don't vary through the sum
                Bk = np.exp(-np.exp(llaG)*np.exp(alpha@(X.T)))
                laM = np.exp(llaL)*np.outer(np.exp(beta@(X.T)),np.exp(gamma@(X.T)))
                laM *= m**eta

                BB = np.zeros((r,r)) # To be the Ball matrix
                for jd in range(0,r):
                    jstr = format(jd,'0' + str(m) + 'b')
                    j = np.array([int(jstr[x]) for x in range(0,len(jstr))])
                    for omd in range(0,jd+1):
                        omstr = format(omd,'0' + str(m) + 'b')
                        om = np.array([int(omstr[x]) for x in range(0,len(omstr))])
                        if np.all(om<=j):
                            BB[jd,omd] = 1.0/np.prod((phi((1-j)@laM,logtheta)**om)*(Bk**(1-j)))
                nlv[i] = -np.log(LA.solve(BB,np.ones(r))[-1])
        nll = np.sum(nlv)
        #nll += 7.4*np.sum(x**2) # Add a Ridge if needed
        nll += np.sum(x**2) # Add a Ridge if needed
        return nll
    except:
        nll = np.inf
        return nll

In [20]:
# Indicative parameters - to do, add bounds and mulitple restarts
x0 = np.array([
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
])
mynll(x0)

2899.9441617670172

In [21]:
bb = np.array([
    [-5.,0.],
    [-5.,0.],
    [-10.,10.],
    [-10.,10.],
    [-3.,3.],
    [-3.,3.],
    [-3.,3.],
])

In [22]:
def callbackF(x, x2=0., x3=0.):
    print('Evaluated at [{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}]: {:.8f}'.format(
        x[0],x[1],x[2],x[3],x[4],x[5],x[6],mynll(x)))

In [23]:
# First try from (essentially) the origin 
foutstore = []
fout = op.minimize(mynll,x0,method='TNC',callback=callbackF,bounds=bb,options={'maxiter' : 10000})
#xhat = fout.x
foutstore.append(fout)

Evaluated at [-0.063,-1.583,0.570,-10.000,0.243,0.201,-0.006]: 831.81659365
Evaluated at [0.000,-1.821,0.398,-10.000,0.043,0.239,0.201]: 700.09734338
Evaluated at [0.000,-3.053,0.466,-10.000,-2.052,0.236,0.203]: 438.53653095
Evaluated at [0.000,-3.443,0.323,-10.000,-2.526,0.221,0.185]: 428.88612235
Evaluated at [0.000,-3.833,0.179,-9.203,-3.000,0.205,0.168]: 417.03844170
Evaluated at [0.000,-3.228,1.170,-3.954,-3.000,-1.179,0.941]: 355.63364113
Evaluated at [0.000,-3.228,1.170,-3.954,-3.000,-1.179,0.941]: 355.63364113
Evaluated at [0.000,-3.695,1.447,-0.713,-3.000,-1.917,1.312]: 338.67088573
Evaluated at [0.000,-3.625,1.401,-1.163,-2.961,-1.793,1.250]: 337.87577110
Evaluated at [0.000,-3.637,1.377,-0.694,-2.918,-1.708,1.224]: 336.46909243
Evaluated at [-1.078,-3.455,1.114,-0.022,-2.721,-0.891,0.901]: 332.47341010
Evaluated at [-1.068,-3.440,1.108,-0.159,-2.726,-0.877,0.892]: 332.35602233
Evaluated at [-1.053,-3.448,1.111,-0.100,-2.723,-0.883,0.895]: 332.31674995
Evaluated at [-0.394,-3

In [24]:
# Because box bounded, try multiple restarts with stable algorithm
np.random.seed(46)
nrestarts = 20
for k in range(0,nrestarts):
    nll0 = np.nan
    while (np.isnan(nll0)) or (np.isinf(nll0)):
        xx0 = np.random.uniform(bb[:,0],bb[:,1])
        nll0 = mynll(xx0)
    try:
        print('Starting at:')
        print(xx0)
        print(nll0)
        fout = op.minimize(mynll,xx0,
                           bounds=bb,method='TNC',callback=callbackF,
                           options={'maxiter' : 1000, 'ftol' : 1e-9})
        print('Found:')
        print(fout.x)
        print('')
        foutstore.append(fout)
    except:
        k -= 1

Starting at:
[-1.08083825 -1.82583147 -5.01913817  5.16151729 -1.12153839  2.62342414
 -2.74280731]
1266.5953531446269
Evaluated at [-2.989,-2.877,-5.354,-1.649,-1.224,0.819,-2.559]: 414.15762030
Evaluated at [-2.783,-3.814,-5.178,-0.811,-1.347,0.961,-2.379]: 382.83297172
Evaluated at [-2.552,-3.985,-5.028,0.375,-1.359,1.131,-2.296]: 366.77769429
Evaluated at [-2.378,-3.501,-0.799,-0.568,-1.269,0.854,-2.126]: 344.28538800
Evaluated at [-0.958,-3.789,-1.678,-0.313,-1.096,0.385,-1.499]: 328.20265682
Evaluated at [-0.479,-3.634,0.527,-0.788,-1.148,0.113,-1.155]: 322.80452584
Evaluated at [-0.531,-3.673,-0.003,-0.677,-1.112,0.149,-1.107]: 322.39549528
Evaluated at [-0.519,-3.641,0.490,-0.777,-1.138,0.086,-0.561]: 321.75993837
Evaluated at [-0.587,-3.683,-0.070,-0.658,-1.097,0.113,-0.315]: 321.30715107
Evaluated at [-0.588,-3.672,0.110,-0.694,-1.105,0.086,-0.027]: 321.20056689
Evaluated at [-0.589,-3.674,0.086,-0.689,-1.104,0.089,-0.062]: 321.19693803
Evaluated at [-0.587,-3.679,0.074,-0.61

Evaluated at [-0.597,-3.687,0.016,-0.620,-1.016,-0.334,-0.090]: 320.80188167
Evaluated at [-0.595,-3.687,0.017,-0.624,-1.013,-0.335,-0.090]: 320.80168413
Evaluated at [-0.595,-3.687,0.017,-0.625,-1.013,-0.336,-0.090]: 320.80168096
Evaluated at [-0.595,-3.687,0.017,-0.625,-1.013,-0.336,-0.090]: 320.80168095
Evaluated at [-0.595,-3.687,0.017,-0.625,-1.013,-0.336,-0.090]: 320.80168094
Evaluated at [-0.595,-3.687,0.017,-0.625,-1.013,-0.336,-0.090]: 320.80168094
Found:
[-0.59503129 -3.68698432  0.01744772 -0.62462571 -1.01254931 -0.33555281
 -0.08978882]

Starting at:
[-0.66459302 -3.38167087 -7.91851205  6.04216094 -0.6310083   0.76660853
 -2.78406874]
861.9642085738585
Evaluated at [-3.618,-3.552,-6.436,2.792,-0.680,-0.595,-2.758]: 394.53974443
Evaluated at [-3.707,-3.825,-6.381,2.577,-0.883,-0.631,-2.440]: 391.56484760
Evaluated at [-3.695,-3.484,-6.292,1.902,-1.373,-0.636,-0.759]: 382.84028338
Evaluated at [-3.622,-3.648,-6.244,1.427,-1.708,-0.616,0.547]: 380.06430089
Evaluated at [-3.5

Evaluated at [-0.548,-3.685,0.083,-0.643,-0.997,-0.372,0.024]: 320.82043506
Evaluated at [-0.554,-3.683,0.024,-0.662,-1.002,-0.372,0.017]: 320.81337884
Evaluated at [-0.554,-3.673,0.039,-0.656,-1.010,-0.368,-0.058]: 320.80698249
Evaluated at [-0.555,-3.676,0.033,-0.656,-1.008,-0.369,-0.042]: 320.80638104
Evaluated at [-0.557,-3.676,0.014,-0.655,-1.010,-0.368,-0.045]: 320.80602263
Evaluated at [-0.556,-3.677,0.018,-0.655,-1.008,-0.369,-0.039]: 320.80592374
Evaluated at [-0.557,-3.677,0.016,-0.655,-1.009,-0.369,-0.040]: 320.80591211
Evaluated at [-0.560,-3.677,0.040,-0.655,-1.009,-0.369,-0.041]: 320.80511888
Evaluated at [-0.567,-3.675,0.024,-0.653,-1.014,-0.367,-0.061]: 320.80405982
Evaluated at [-0.568,-3.675,0.026,-0.653,-1.014,-0.367,-0.061]: 320.80404891
Evaluated at [-0.568,-3.675,0.027,-0.653,-1.014,-0.367,-0.061]: 320.80404890
Found:
[-0.56792714 -3.67518965  0.0265004  -0.65343006 -1.01367408 -0.36739398
 -0.06133402]

Starting at:
[-3.50046075 -4.7617339  -2.55577844 -4.7516960

Evaluated at [-0.570,-3.681,0.054,-0.641,-1.058,-0.387,-0.060]: 320.80594828
Evaluated at [-0.568,-3.681,0.055,-0.637,-1.056,-0.387,-0.062]: 320.80553890
Evaluated at [-0.568,-3.681,0.055,-0.635,-1.056,-0.387,-0.066]: 320.80550051
Evaluated at [-0.568,-3.681,0.055,-0.635,-1.056,-0.387,-0.066]: 320.80550037
Evaluated at [-0.568,-3.681,0.055,-0.635,-1.056,-0.387,-0.067]: 320.80549996
Evaluated at [-0.568,-3.681,0.055,-0.635,-1.056,-0.387,-0.068]: 320.80549271
Evaluated at [-0.570,-3.681,0.056,-0.635,-1.057,-0.388,-0.075]: 320.80542930
Evaluated at [-0.566,-3.681,0.053,-0.643,-1.047,-0.386,-0.054]: 320.80442211
Evaluated at [-0.568,-3.681,0.052,-0.647,-1.043,-0.385,-0.066]: 320.80384392
Evaluated at [-0.568,-3.681,0.052,-0.647,-1.043,-0.385,-0.066]: 320.80384332
Evaluated at [-0.568,-3.681,0.052,-0.647,-1.043,-0.385,-0.066]: 320.80384312
Evaluated at [-0.568,-3.681,0.053,-0.647,-1.043,-0.385,-0.067]: 320.80384288
Evaluated at [-0.568,-3.681,0.052,-0.647,-1.043,-0.385,-0.066]: 320.80381668

Evaluated at [-0.639,-3.678,-0.003,-0.593,-1.034,-0.324,-0.129]: 320.81112935
Found:
[-6.38792767e-01 -3.67838342e+00 -2.64949523e-03 -5.92677177e-01
 -1.03411350e+00 -3.23813863e-01 -1.28746737e-01]

Starting at:
[-0.68708015 -4.54957372 -7.02022197  3.45149927  2.87395155 -2.86065897
  2.44629415]
919.1484956903099
Evaluated at [-2.456,-4.769,-2.189,2.121,2.300,-2.857,1.206]: 417.74894237
Evaluated at [-3.343,-4.420,0.439,1.356,-0.333,-2.452,0.698]: 356.43125966
Evaluated at [-2.988,-4.322,-0.565,1.602,-0.384,-2.343,0.892]: 350.50638762
Evaluated at [-2.803,-4.240,-1.076,1.714,-0.513,-2.251,0.983]: 349.50346565
Evaluated at [-2.619,-4.147,-1.598,1.702,-0.652,-2.126,0.729]: 348.30018363
Evaluated at [-2.789,-4.213,-1.136,1.485,-0.541,-2.181,0.334]: 345.80577692
Evaluated at [-2.574,-4.055,-1.788,0.911,-0.744,-1.884,-1.499]: 342.26782031
Evaluated at [-2.158,-3.811,-2.911,0.458,-0.743,-1.636,-1.395]: 339.78686452
Evaluated at [-1.919,-3.808,-2.926,0.451,-0.744,-1.632,-1.393]: 339.15294

...

Evaluated at [-3.008,-3.538,-0.219,1.171,-1.168,-0.123,-1.752]: 335.91345630
Evaluated at [-2.597,-3.832,0.131,0.948,-1.206,-0.137,-1.700]: 332.66957142
Evaluated at [-2.610,-3.835,0.142,0.941,-1.073,-0.138,-1.698]: 332.61321449
Evaluated at [-2.598,-3.845,0.155,0.932,-1.047,-0.138,-1.696]: 332.60779629
Evaluated at [-2.575,-3.862,0.177,0.917,-1.004,-0.139,-1.693]: 332.59280220
Evaluated at [-2.038,-3.870,0.211,0.820,-0.798,-0.143,-1.680]: 331.02693593
Evaluated at [-2.078,-3.808,0.129,0.868,-0.946,-0.140,-1.691]: 330.83435861
Evaluated at [-2.097,-3.796,0.114,0.878,-0.977,-0.139,-1.694]: 330.82697752
Evaluated at [-2.253,-3.726,0.042,0.883,-1.133,-0.137,-1.704]: 330.61309221
Evaluated at [-2.147,-3.842,0.243,0.652,-0.755,-0.147,-1.672]: 329.84598054
Evaluated at [-2.214,-3.815,0.220,0.641,-0.809,-0.146,-1.675]: 329.74643403
Evaluated at [-2.232,-3.813,0.227,0.617,-0.800,-0.147,-1.674]: 329.73386534
Evaluated at [-0.696,-3.771,0.146,-0.243,-0.872,-0.159,-1.608]: 324.58430838
Evaluated 

Evaluated at [-0.612,-3.682,0.026,-0.617,-1.021,-0.371,-0.028]: 320.80316315
Evaluated at [-0.611,-3.682,0.027,-0.618,-1.020,-0.372,-0.028]: 320.80315565
Evaluated at [-0.606,-3.681,0.020,-0.615,-1.026,-0.359,-0.029]: 320.80228789
Evaluated at [-0.600,-3.683,0.023,-0.620,-1.015,-0.364,-0.029]: 320.80180703
Evaluated at [-0.599,-3.683,0.023,-0.619,-1.015,-0.362,-0.029]: 320.80179254
Evaluated at [-0.599,-3.683,0.023,-0.619,-1.015,-0.362,-0.029]: 320.80179253
Evaluated at [-0.599,-3.683,0.023,-0.619,-1.015,-0.362,-0.029]: 320.80179253
Found:
[-0.59944762 -3.68266526  0.02269143 -0.61936576 -1.01531148 -0.36246397
 -0.02948061]

Starting at:
[-3.80074058 -3.96526117 -8.32967381 -9.73736909  0.98780121 -0.98206783
  2.65622619]
561.6209663071907
Evaluated at [-3.313,-3.735,0.738,1.391,-0.074,-0.941,2.521]: 346.90066719
Evaluated at [-3.343,-3.803,0.187,0.716,-0.182,-0.944,2.027]: 343.42448713
Evaluated at [-3.311,-3.937,0.765,1.425,-0.727,-0.941,0.338]: 339.31197072
Evaluated at [-3.319,-3

Evaluated at [-0.592,-3.688,0.027,-0.624,-1.024,-0.366,-0.083]: 320.80043824
Evaluated at [-0.592,-3.688,0.026,-0.624,-1.023,-0.367,-0.084]: 320.80042821
Evaluated at [-0.592,-3.688,0.026,-0.624,-1.023,-0.367,-0.084]: 320.80042820
Evaluated at [-0.592,-3.688,0.026,-0.624,-1.023,-0.367,-0.084]: 320.80042820
Found:
[-0.59218923 -3.68838371  0.02590215 -0.6242124  -1.02267011 -0.36652195
 -0.08373526]

Starting at:
[-2.02452975 -3.24084335  5.67443168  6.956104   -2.24344927  2.08531022
 -1.94804439]
438.96562571929564
Evaluated at [-1.875,-3.984,-2.442,-0.134,-2.034,1.900,-1.769]: 351.54580550
Evaluated at [-1.884,-3.873,-2.454,-0.518,-2.032,1.807,-1.731]: 349.12277381
Evaluated at [-1.895,-3.459,-1.048,-0.209,-2.056,1.554,-1.636]: 339.16062043
Evaluated at [-1.875,-3.484,-0.893,0.040,-2.053,1.482,-1.619]: 338.01995807
Evaluated at [-1.705,-3.867,0.109,0.084,-1.992,0.621,-1.401]: 330.57892453
Evaluated at [-1.456,-3.583,0.780,-0.109,-1.906,-0.323,-1.056]: 326.37274242
Evaluated at [-1.49

Evaluated at [-0.604,-3.684,0.028,-0.613,-1.023,-0.363,-0.066]: 320.79986559
Evaluated at [-0.604,-3.684,0.029,-0.613,-1.023,-0.363,-0.065]: 320.79985244
Evaluated at [-0.603,-3.684,0.030,-0.613,-1.024,-0.362,-0.065]: 320.79985064
Evaluated at [-0.603,-3.684,0.030,-0.613,-1.024,-0.362,-0.065]: 320.79984886
Evaluated at [-0.603,-3.684,0.029,-0.613,-1.024,-0.362,-0.065]: 320.79984846
Evaluated at [-0.603,-3.684,0.029,-0.613,-1.024,-0.362,-0.065]: 320.79984842
Found:
[-0.60344317 -3.68406518  0.02932995 -0.61258265 -1.02398944 -0.36225563
 -0.06508788]

Starting at:
[-4.98650948 -4.64486441 -3.22408587 -3.66012014  0.16363433 -1.19011819
  1.25917591]
448.596249740338
Evaluated at [-4.630,-3.470,-0.565,-0.545,0.011,-1.089,1.212]: 382.77763246
Evaluated at [-4.566,-3.260,-0.089,0.012,-0.016,-1.071,1.204]: 380.51106226
Evaluated at [-4.557,-3.228,-0.017,0.097,-0.021,-1.067,1.202]: 380.45621652
Evaluated at [-4.561,-3.248,-0.064,0.042,-0.018,-1.068,1.203]: 380.44026931
Evaluated at [-4.530,-

Evaluated at [-0.585,-3.698,-0.192,-0.597,-0.996,-0.407,-0.117]: 320.88525296
Evaluated at [-0.605,-3.702,-0.195,-0.615,-1.015,-0.407,-0.116]: 320.87164306
Evaluated at [-0.606,-3.690,-0.116,-0.646,-1.015,-0.405,-0.114]: 320.83590296
Evaluated at [-0.604,-3.678,-0.030,-0.609,-1.015,-0.402,-0.112]: 320.80881382
Evaluated at [-0.612,-3.686,0.004,-0.603,-1.019,-0.393,-0.101]: 320.80393795
Evaluated at [-0.613,-3.683,0.029,-0.601,-1.020,-0.392,-0.099]: 320.80305065
Evaluated at [-0.608,-3.681,0.014,-0.602,-1.020,-0.385,-0.097]: 320.80224909
Evaluated at [-0.607,-3.680,0.022,-0.602,-1.020,-0.383,-0.096]: 320.80207198
Evaluated at [-0.607,-3.680,0.024,-0.602,-1.020,-0.383,-0.096]: 320.80206524
Evaluated at [-0.608,-3.681,0.027,-0.610,-1.020,-0.382,-0.096]: 320.80135182
Evaluated at [-0.608,-3.681,0.028,-0.610,-1.020,-0.382,-0.096]: 320.80134526
Evaluated at [-0.608,-3.681,0.028,-0.609,-1.020,-0.382,-0.096]: 320.80134439
Evaluated at [-0.608,-3.681,0.029,-0.610,-1.020,-0.382,-0.096]: 320.8013

In [25]:
foutstore

[     fun: 320.80809678726365
      jac: array([-0.20688162, -0.21703386,  0.03097398, -0.06239134,  0.10978738,
         0.01946887, -0.14636043])
  message: 'Converged (|f_n-f_(n-1)| ~= 0)'
     nfev: 414
      nit: 66
   status: 1
  success: True
        x: array([-0.62268069, -3.68692129,  0.03445048, -0.60148146, -1.004808  ,
        -0.35094256, -0.13870192]),      fun: 320.79915736796187
      jac: array([-0.00011937, -0.00209752, -0.00013074,  0.00024443, -0.00065938,
        -0.00135287, -0.00065938])
  message: 'Converged (|f_n-f_(n-1)| ~= 0)'
     nfev: 245
      nit: 34
   status: 1
  success: True
        x: array([-0.59086645, -3.68248939,  0.02798056, -0.62473457, -1.0234201 ,
        -0.36251107, -0.07563111]),      fun: 320.80168093823886
      jac: array([-0.01447802, -0.24910491, -0.02523848, -0.00373461,  0.0549619 ,
         0.09655423, -0.03020659])
  message: 'Converged (|f_n-f_(n-1)| ~= 0)'
     nfev: 429
      nit: 74
   status: 1
  success: True
        x: arr

In [26]:
ff = np.inf*np.ones(len(foutstore))
for i in range(0,len(foutstore)): # In case of crash
    if foutstore[i].success:
        ff[i] = foutstore[i].fun

In [27]:
xhat = foutstore[ff.argmin()].x
print(xhat)

[-0.59086645 -3.68248939  0.02798056 -0.62473457 -1.0234201  -0.36251107
 -0.07563111]


In [28]:
pn = len(x0)
delta = 1e-2 # This will need some tuning, but here set at sqrt(default delta in optimiser)
dx = delta*xhat
ej = np.zeros(pn)
ek = np.zeros(pn)
Hinv = np.zeros((pn,pn))
for j in tqdm(range(0,pn)):
    ej[j] = dx[j]
    for k in range(0,j):
        ek[k] = dx[k]
        Hinv[j,k] = mynll(xhat+ej+ek) - mynll(xhat+ej-ek) - mynll(xhat-ej+ek) + mynll(xhat-ej-ek)
        ek[k] = 0.
    Hinv[j,j] = - mynll(xhat+2*ej) + 16*mynll(xhat+ej) - 30*mynll(xhat) + 16*mynll(xhat-ej) - mynll(xhat-2*ej)
    ej[j] = 0.
Hinv += np.triu(Hinv.T,1)
Hinv /= (4.*np.outer(dx,dx) + np.diag(8.*dx**2)) # TO DO: replace with a chol ...
covmat = LA.inv(0.5*(Hinv+Hinv.T))
stds = np.sqrt(np.diag(covmat))
stds

array([0.4538739 , 0.12676303, 0.6668649 , 0.41420348, 0.41597157,
       0.52920231, 0.67513573])

In [29]:
print('Baseline probability of infection from outside is {:.1f} ({:.1f},{:.1f}) %'.format(
    100.*(1.-np.exp(-np.exp(xhat[1]))),
    100.*(1.-np.exp(-np.exp(xhat[1]-1.96*stds[1]))),
    100.*(1.-np.exp(-np.exp(xhat[1]+1.96*stds[1]))),
    ))

# phi gets bigger as xhat[1] gets smaller and bigger as xhat[2] gets bigger
# 'Safest' method is Monte Carlo - sample 

mymu = xhat[[0,2,3]]
mySig = covmat[[0,2,3],:][:,[0,2,3]]
m = 4000

for k in range(2,7):
    sarvec = np.zeros(m)
    for i in range(0,m):
        uu = np.random.multivariate_normal(mymu,mySig)
        eta = (4./np.pi)*np.arctan(uu[2])
        sarvec[i] = 100.*(1.-phi(np.exp(uu[0])*(k**eta),uu[1]))

    eta = (4./np.pi)*np.arctan(xhat[3])
    print('HH size {:d} baseline pairwise infection probability is {:.1f} ({:.1f},{:.1f}) %'.format(
        k,
        100.*(1.-phi(np.exp(xhat[0])*(k**eta),xhat[2])),
        np.percentile(sarvec,2.5),
        np.percentile(sarvec,97.5),
        ))  


print('Relative external exposure for <=20yo {:.1f} ({:.1f},{:.1f}) %'.format(
    100.*np.exp(xhat[4]),
    100.*np.exp(xhat[4]-1.96*stds[4]),
    100.*np.exp(xhat[4]+1.96*stds[4]),
    ))
print('Relative susceptibility for <=20yo {:.1f} ({:.1f},{:.1f}) %'.format(
    100.*np.exp(xhat[5]),
    100.*np.exp(xhat[5]-1.96*stds[5]),
    100.*np.exp(xhat[5]+1.96*stds[5]),
    ))
print('Relative transmissibility for <=20yo {:.1f} ({:.1f},{:.1f}) %'.format(
    100.*np.exp(xhat[6]),
    100.*np.exp(xhat[6]-1.96*stds[6]),
    100.*np.exp(xhat[6]+1.96*stds[6]),
    ))

Baseline probability of infection from outside is 2.5 (1.9,3.2) %
HH size 2 baseline pairwise infection probability is 25.2 (16.4,38.3) %
HH size 3 baseline pairwise infection probability is 20.2 (13.5,31.2) %
HH size 4 baseline pairwise infection probability is 17.1 (11.1,29.4) %
HH size 5 baseline pairwise infection probability is 15.0 (9.2,29.5) %
HH size 6 baseline pairwise infection probability is 13.4 (7.8,30.5) %
Relative external exposure for <=20yo 35.9 (15.9,81.2) %
Relative susceptibility for <=20yo 69.6 (24.7,196.3) %
Relative transmissibility for <=20yo 92.7 (24.7,348.2) %
